In [1]:
import pandas as pd
df = pd.read_excel("DATA_Kiss_count_gender_and_IQ.xlsx")

In [2]:
print(df.columns)

Index(['Name', 'Gender', 'IQ', 'Kiss Count', 'Age of First Kiss'], dtype='object')


In [5]:
import pandas as pd
from scipy.stats import pearsonr, ttest_ind, t


df = pd.read_excel("DATA_Kiss_count_gender_and_IQ.xlsx")
df['Gender'] = df['Gender'].str.strip().str.lower()
df['GenderFlag'] = df['Gender'].apply(lambda x: 1 if x == 'male' else 0)
df = df.dropna(subset=['IQ', 'Kiss Count', 'Age of First Kiss', 'GenderFlag'])

def r_to_t(r, n):
    return r * ( (n - 2) ** 0.5 ) / ((1 - r ** 2) ** 0.5)

def t_to_p(t_val, dfree):
    return 2 * t.sf(abs(t_val), dfree)

results = []
n = len(df)

variables = [
    ('IQ', df['IQ']),
    ('Age of First Kiss', df['Age of First Kiss']),
    ('Gender (male=1)', df['GenderFlag'])
]

for var_name, series in variables:
    r, corr_p = pearsonr(df['Kiss Count'], series)
    t_val = r_to_t(r, n)
    p_from_t = t_to_p(t_val, n - 2)

    if var_name.startswith('Gender'):
        group_m = df[df['GenderFlag'] == 1]['Kiss Count']
        group_f = df[df['GenderFlag'] == 0]['Kiss Count']
        t_stat, p_ttest = ttest_ind(group_m, group_f)
    else:
        t_stat, p_ttest = '', ''

    results.append({
        'Variable': var_name,
        'Pearson r': round(r, 6),
        'Correlation p (from r)': round(corr_p, 6),
        't (from r)': round(t_val, 6),
        'Correlation p (from t)': round(p_from_t, 6),
        't-test value (group diff)': t_stat if t_stat == '' else round(t_stat, 6),
        't-test p (group diff)': p_ttest if p_ttest == '' else round(p_ttest, 6)
    })

result_df = pd.DataFrame(results)
print(result_df)


            Variable  Pearson r  Correlation p (from r)  t (from r)  \
0                 IQ   0.110812                0.118262    1.568927   
1  Age of First Kiss  -0.230764                0.001011   -3.337208   
2    Gender (male=1)  -0.003463                0.961185   -0.048728   

   Correlation p (from t) t-test value (group diff) t-test p (group diff)  
0                0.118262                                                  
1                0.001011                                                  
2                0.961185                 -0.048728              0.961185  


In [6]:
import pandas as pd
from scipy.stats import pearsonr, t
from itertools import combinations

df = pd.read_excel("DATA_Kiss_count_gender_and_IQ.xlsx")
df['Gender'] = df['Gender'].str.strip().str.lower()
df['GenderFlag'] = df['Gender'].apply(lambda x: 1 if x == 'male' else 0)
df = df.dropna(subset=['IQ', 'Kiss Count', 'Age of First Kiss', 'GenderFlag'])

def r_to_t(r, n):
    return r * ((n - 2) ** 0.5) / ((1 - r ** 2) ** 0.5)

def t_to_p(t_val, dfree):
    return 2 * t.sf(abs(t_val), dfree)

cols = ['IQ', 'Kiss Count', 'Age of First Kiss', 'GenderFlag']
n = len(df)
results = []

for var1, var2 in combinations(cols, 2):
    x = df[var1]
    y = df[var2]
    
    r_val, p_corr = pearsonr(x, y)
    t_val = r_to_t(r_val, n)
    p_from_t = t_to_p(t_val, n - 2)

    results.append({
        'Var A': var1,
        'Var B': var2,
        'Pearson r': round(r_val, 6),
        'Pearson p': round(p_corr, 6),
        't (from r)': round(t_val, 6),
        'p (from t)': round(p_from_t, 6)
    })

for var1, var2 in combinations(cols, 2):
    row = next(item for item in results if item['Var A'] == var1 and item['Var B'] == var2)
    results.append({
        'Var A': var2,
        'Var B': var1,
        'Pearson r': row['Pearson r'],
        'Pearson p': row['Pearson p'],
        't (from r)': row['t (from r)'],
        'p (from t)': row['p (from t)']
    })

result_df = pd.DataFrame(results)
result_df = result_df.sort_values(by=['Var A', 'Var B'])
print(result_df)


                Var A              Var B  Pearson r  Pearson p  t (from r)  \
5   Age of First Kiss         GenderFlag  -0.194026   0.005906   -2.783069   
7   Age of First Kiss                 IQ  -0.041842   0.556346   -0.589279   
9   Age of First Kiss         Kiss Count  -0.230764   0.001011   -3.337208   
11         GenderFlag  Age of First Kiss  -0.194026   0.005906   -2.783069   
8          GenderFlag                 IQ   0.128213   0.070401    1.819135   
10         GenderFlag         Kiss Count  -0.003463   0.961185   -0.048728   
1                  IQ  Age of First Kiss  -0.041842   0.556346   -0.589279   
2                  IQ         GenderFlag   0.128213   0.070401    1.819135   
0                  IQ         Kiss Count   0.110812   0.118262    1.568927   
3          Kiss Count  Age of First Kiss  -0.230764   0.001011   -3.337208   
4          Kiss Count         GenderFlag  -0.003463   0.961185   -0.048728   
6          Kiss Count                 IQ   0.110812   0.118262  